In [31]:
import yaml
import itertools
import re
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

from src.io import read_yaml_config, read_fasta_as_dict, get_augustus_proteins, read_predictions
from src.visual import visualize_predictions, visualize_single_prediction


warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
config = read_yaml_config("config.yaml")

In [3]:
TARGET_SPECIES = "Ganaspis_brasiliensis"

In [4]:
TARGET_SPECIES = TARGET_SPECIES.replace(" ", "_")
genome_file = Path(config['data_locations']['genomes_dir']) / f'{TARGET_SPECIES}.fna'
proteome_file = Path(config['data_locations']['proteomes_dir']) / f'{TARGET_SPECIES}.faa'
predictions_file = Path(config['data_locations']['raw_predictions_dir']) / f'{TARGET_SPECIES}.tsv'
genes_file = Path(config['data_locations']['gene_predictions_dir']) / f'{TARGET_SPECIES}.fna'
annotation_file = Path(config['data_locations']['gene_annotations_dir']) / f'{TARGET_SPECIES}.gff'
false_positives_file = Path(config['data_locations']['false_positives_dir']) / f'{TARGET_SPECIES}.faa'

In [5]:
genome_map = read_fasta_as_dict(genome_file)
proteome_map = read_fasta_as_dict(proteome_file)
genes_map = read_fasta_as_dict(genes_file)
fp_map = read_fasta_as_dict(false_positives_file)
predictions_data = read_predictions(predictions_file)

In [6]:
predictions_data.head(15)

,record_id,record_description,pos_count,sequence,prediction_mask,frame
0,RJVV01031371.1_start=0_end=30000_frame=+2,RJVV01031371.1_start=0_end=30000_frame=+2 RJVV...,66,FQK*DWFGIEKVRNSSIFDITLIQTNLLLLAGMTFI**FRDIRYFL...,0000000000000000000000000000000000000000000000...,2
1,RJVV01031371.1_start=0_end=30000_frame=+3,RJVV01031371.1_start=0_end=30000_frame=+3 RJVV...,66,FKNEIGLALKKFEIPVYLISL*SKPIYFFLPG*HLFNDFVILDIFY...,0000000000000000000000000000000000000000000000...,3
2,RJVV01095373.1_start=0_end=30000_frame=+1,RJVV01095373.1_start=0_end=30000_frame=+1 RJVV...,37,ILNFQRVRREPGRPGNMPRPKPIPIRPRPPHPVCCFNNLKFIIFHK...,1111111111111111111111111111111111111000000000...,1
3,RJVV01183868.1_start=0_end=30000_frame=-2,RJVV01183868.1_start=0_end=30000_frame=-2 RJVV...,37,VENQAVQEICHDLNQFQLDLDHHTQ*VLHLLKMNNN****QKKK*C...,0000000000000000000000000000000000000000000000...,2
4,RJVV01096013.1_start=0_end=30000_frame=-1,RJVV01096013.1_start=0_end=30000_frame=-1 RJVV...,34,ILNFQRVRREPGRPGNMPRPKPIPIRPRPPHPVSSFNNP*FIIFHS...,1111111111111111111111111111111111000000000000...,1
5,RJVV01183868.1_start=0_end=30000_frame=-1,RJVV01183868.1_start=0_end=30000_frame=-1 RJVV...,33,RREPGRPGNMPRPKPIPIRPRPPHPVSTTFIENE**LIVITKKEIM...,1111111111111111111111111111111110000000000000...,1
6,RJVV01026129.1_start=0_end=30000_frame=+1,RJVV01026129.1_start=0_end=30000_frame=+1 RJVV...,31,KGLEFSNICNILLDRRASFKDKFSIKK*FSKKMNLCGFCTNCLNLK...,0000000000000000000000000000000000000000000000...,1
7,RJVV01467345.1_start=0_end=30000_frame=+1,RJVV01467345.1_start=0_end=30000_frame=+1 RJVV...,30,RVRREPGRPGNMPRPKPIPIRPRPPHPVSTIFIQILNIN****QEK...,1111111111111111111111111111101000000000000000...,1
8,RJVV01026129.1_start=0_end=30000_frame=+3,RJVV01026129.1_start=0_end=30000_frame=+3 RJVV...,29,GFRV**HM*YFTRS*SFI*R*VFHKKVIFEKNELVRFLHQLPQFKN...,0000000000000000000000000000000000000000000000...,3
9,RJVV01527581.1_start=0_end=30000_frame=+3,RJVV01527581.1_start=0_end=30000_frame=+3 RJVV...,25,PGNMPRPKPIPIRPRPPHPVSTIFIENE**LIVITRKEIMHIYIFI...,1111111111111111111111111000000000000000000000...,3


In [7]:
SCORE_THRESHOLD = 60

selected_data = predictions_data.query("pos_count > @SCORE_THRESHOLD")

In [42]:
2 % 1

0

# Visualize predictions

In [10]:
SHOW_NUCLEOTIDE = True
SHOW_TRANSLATED = True
SHOW_MISSING_FRAMES = True
NUCL_VISUAL_OFFSET = np.array((1500, 500))
AA_VISUAL_OFFSET = NUCL_VISUAL_OFFSET // 3


visualize_predictions(
    selected_data,
    genome_map=genome_map,
    proteome_map=proteome_map,
    show_translated=SHOW_TRANSLATED,
    show_nucleotide=SHOW_NUCLEOTIDE,
    show_missing_frames=SHOW_MISSING_FRAMES,
    nucl_visual_offset=NUCL_VISUAL_OFFSET
)

RJVV01031371.1_start=0_end=30000_frame=+2 Score=66 Fragment_start=1139_end=1968
YYLSKFDENYSCSQILKKIHKMLNNYYLITNFESKNFFFYAQLGLIELGKIKFNNIFIRREKPFSII*GSVCQWVEHP*QVYVNNYIQILIVVIFSEPG*NGED*CFLNRSTICICSFICSS*TRTCSGW*TFAP*TTSNQTPSSTSCKSIIFFIKNLMFNNF*RKNIIRSHNYRIFYSILT*YKKLSKYCESYIQINNFWYNVRGNY*K*CKSIH*TNYYINYK*LK*KQNIQ**IENQIINIKSSLYLKRLRREAEGLEEDVAEVETDEVEESAVALDVRTR*NY**KKYSYDIFYFL*LKQIKYKQ*HYKN*NLL*LNTPPVFKLKIQVQIY*FKIKVKKKLIIIK*FIVINKYYAFLESSSRTRSSRKYATS*TNSN*TSTPTPSEFYILLKINNNYNIFMHLLYE*TDKAGSKLIIFFSSFSVFVVKLKNLKQKTFCL*R*KPRSSFL*KYNYNKILFKKI*I*FKITTKIKYFLYEEAIFKNKQLSIIIQFLIIILNFQRVRREPGRPGNMPRPKPIPIRPRPPHPLYNYKYTLLVKKMN*FELFHLSASSS*S*RT*SRRP*S**SFAFRGKKTKLFYCMKI***QNFI*EKKIYF*NKTNKKYFKN*QLAKIIQFLIIILNF*RVRREPGRPGNMPRPKPIPIRPRPPHPVSTAFIENK**LIVIT*IEIMHIYIIISTLFLSKNKLILTFSSFSVFAEKLKNLK*KTLKQMMFCL*R*EKIDFLLYENIVITKLYLRKFKFNLKLQQK*TIGYYNSIFNYNLELSESTSRTRPSRKYATTQTNSN*T*TTTPSEYYIY*K*IIINRKEIMHIYIIVSILF
RJVV01031371.1_start=0_end=30000_frame=+3 Score=66 Fragment_start=775_end=1986
EDGDSPSFLAR

In [23]:
SHOW_NUCLEOTIDE = True
SHOW_TRANSLATED = True
SHOW_MISSING_FRAMES = False
NUCL_VISUAL_OFFSET = np.array((1500, 500))
AA_VISUAL_OFFSET = (1, 1) #NUCL_VISUAL_OFFSET // 3


visualize_predictions(
    selected_data,
    genome_map=genome_map,
    proteome_map=proteome_map,
    show_translated=SHOW_TRANSLATED,
    show_nucleotide=SHOW_NUCLEOTIDE,
    show_missing_frames=SHOW_MISSING_FRAMES,
    nucl_offset=NUCL_VISUAL_OFFSET,
    aa_offset=AA_VISUAL_OFFSET
)

RJVV01031371.1_start=0_end=30000_frame=+2 Score=66 Fragment_start=0_end=165
IILNFQRVRREPGRPGNMPRPKPIPIRPRPPHPLYNYKYTLLVKKMN*FELFHLSASSS*S*RT*SRRP*S**SFAFRGKKTKLFYCMKI***QNFI*EKKIYF*NKTNKKYFKN*QLAKIIQFLIIILNF*RVRREPGRPGNMPRPKPIPIRPRPPHPVSTAFI
RJVV01031371.1_start=0_end=30000_frame=+3 Score=66 Fragment_start=0_end=671
RPPPIKPRPPHPVSQLFFLLKI*CSIISEEKI*FVHIIIGFFILYSHSIKNYLNTAKVTFR*IIFGIMCVVIIKNNVNLFIKQIII*IINN*NKNKIFNNKLKIK*LILNHLFI*SAFVVKLKALKKM*PKLKLMKWKNLQLL*M*ELDETINEKNIVMIYSIFYD*NR*NINNNITRIKTYYD*ILLQFLN*KYKFKFTDSKLK*KKN*L**NNL*L*INIMHF*RVRREPGRPGNMPRPKPIPIRPRPPHPVSSIFY*KLIIIIIYLCIYYMNKLIRPGVN*LYFFHLSASSS*S*RT*SRRRFAFRGKNHDLLFYENIIITKFYLRKFKFNLKLQQK*NTSSMKKQYSKINNCLL*SNF*L*F*TFREFVENQAVQEICHDLNQFQLDLDHHTLYIIISTLFLSKK*INLNFFIFQRLRREAEELEAEDLEADEVLPLEVKKPSSSIV*KYNNNKILFKKKKFIFKIKQTKNILKINNWLK*SNS*L*F*TFREFVENQAVQEICHDLNQFQLDLDHHTQ*VLHLLKINNN****HE*K*CIFI*L*VHSSYQKIN*F*LFHLSASSPRS*RT*SRRP*SR*CFAFRGKKRSISYCMKI*L*QNFI*ENLNLI*NYSKNKQLAIIIPFLIIILNFQRVRREPGRPGNMPRPKPIPIRPRPPHPVSIIF
RJVV010313

# False Positives

## Overview

In [28]:
from src.io import FragmentRecordId
from src.predictions import extract_fp_aa_predictions

In [34]:
fp_rec_ids = {FragmentRecordId(rec_id) for rec_id in fp_map}
fp_predictions = extract_fp_aa_predictions(predictions_data.query("record_id in @fp_rec_ids"), kernel_size=33)
for _, row in fp_predictions.iterrows():
    visualize_single_prediction(row.sequence, row.prediction_mask, caption=f"{row.record_id} Score={row.pos_count}")
    print()

RJVV01491319.1_start=0_end=30000_frame=-1 Score=14 Fragment_start=0_end=33
TIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK

RJVV01518257.1_start=0_end=30000_frame=-2 Score=14 Fragment_start=0_end=33
TIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK

RJVV01002766.1_start=0_end=30000_frame=+2 Score=10 Fragment_start=0_end=43
SLHSASLNPFTLTDFLTFFHFMAPRTYRNFRFSQIRHPGVKSR

RJVV01011221.1_start=0_end=30000_frame=-1 Score=10 Fragment_start=0_end=43
QELRKMNTTRRFAVLHPFFKFIPPRWLHQSQIRTFHDKTVTLI

RJVV01040401.1_start=0_end=30000_frame=+2 Score=8 Fragment_start=0_end=40
ELTLDSEDLKRIFLGFPKNSQEYSREFHDFP*KSHKWPRI

RJVV01502259.1_start=0_end=30000_frame=+1 Score=8 Fragment_start=0_end=25
PIRPRPPHPVSTIFIENE**LIEKK

RJVV01507231.1_start=0_end=30000_frame=-1 Score=7 Fragment_start=0_end=23
NLPGEESDDESIDEGYDSPLDRT

RJVV01380973.1_start=0_end=30000_frame=+3 Score=6 Fragment_start=0_end=41
TIRNVNKKFF*LLLFCFVPRASTKNYREIHHGKFKLLRNKY

RJVV01008589.1_start=0_end=30000_frame=-1 Score=5 Fragment_start=0_end=38
RTLRRVISPVLRRDFTTFFPRAQERDSHGTYRF

## Detailed

In [12]:
TARGET_RECORD_ID = "RJVV01491319.1_start=0_end=30000_frame=-"
SHOW_NUCLEOTIDE = True
SHOW_TRANSLATED = True
SHOW_MISSING_FRAMES = True
NUCL_VISUAL_OFFSET = np.array((1500, 500))
AA_VISUAL_OFFSET = NUCL_VISUAL_OFFSET // 3


fp_selected_data = predictions_data[predictions_data.record_id.astype(str).str.startswith(TARGET_RECORD_ID)]


visualize_predictions(fp_selected_data, genomic_map, proteome_map, show_translated=SHOW_TRANSLATED, show_nucleotide=SHOW_NUCLEOTIDE, show_missing_frames=SHOW_MISSING_FRAMES, nucl_visual_offset=NUCL_VISUAL_OFFSET)

RJVV01491319.1_start=0_end=30000_frame=-1 Score=14 Fragment_start=0_end=89
S*RT*SRRP*SR*RFALRGNKRSIFYCVKI*L*KHFI*EN*NLF*N*NKNKIF*KGTIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK
RJVV01491319.1_start=0_end=30000_frame=-2 Score=0 Fragment_start=0_end=88
AEELEVEDLEADDVLPLEVTKDRFFIV*KYNYKNILFKKIEIYFKIKTKIKYFKKGQLAKIFQFLII*F*TFREFVENQAVQEICHDL
RJVV01491319.1_start=0_end=30000_frame=-3 Score=0 Fragment_start=0_end=88
LKNLK*KTLKQMTFCP*R*QKIDFLLCENIIIKTFYLRKLKFILKLKQK*NILKRDNWLKYSNF*LYNFKLSESSSRTRPSRKYATT*
RJVV01491319.1_start=0_end=30000_frame=- Fragment_start=0_end=267
AGCTGAAGAACTTGAAGTAGAAGACCTTGAAGCAGATGACGTTTTGCCCTTAGAGGTAACAAAAGATCGATTTTTTATTGtgtgaaaatataattataaaaacattttatttaagaaaattgaaatttattttaaaattaaaacaaaaataaaatattttaaaaagggaCAATTGGctaaaatattccaatttttaattatataattttaaactttcagagAGTTCGTCGAGAACCAGGCCGTCCAGGAAATATGCCACGACCTAAA
                                                                                                                        


In [35]:
TARGET_RECORD_ID = "RJVV01491319.1_start=0_end=30000_frame=-"
SHOW_NUCLEOTIDE = True
SHOW_TRANSLATED = True
SHOW_MISSING_FRAMES = True
NUCL_VISUAL_OFFSET = np.array((1500, 500))
AA_VISUAL_OFFSET = NUCL_VISUAL_OFFSET // 3


fp_selected_data = predictions_data[predictions_data.record_id.astype(str).str.startswith(TARGET_RECORD_ID)]


visualize_predictions(fp_selected_data, genome_map=genome_map, proteome_map=proteome_map, show_translated=SHOW_TRANSLATED, show_nucleotide=SHOW_NUCLEOTIDE, show_missing_frames=SHOW_MISSING_FRAMES, nucl_offset=NUCL_VISUAL_OFFSET)

RJVV01491319.1_start=0_end=30000_frame=-1 Score=14 Fragment_start=0_end=89
S*RT*SRRP*SR*RFALRGNKRSIFYCVKI*L*KHFI*EN*NLF*N*NKNKIF*KGTIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK
RJVV01491319.1_start=0_end=30000_frame=-2 Score=0 Fragment_start=0_end=88
AEELEVEDLEADDVLPLEVTKDRFFIV*KYNYKNILFKKIEIYFKIKTKIKYFKKGQLAKIFQFLII*F*TFREFVENQAVQEICHDL
RJVV01491319.1_start=0_end=30000_frame=-3 Score=0 Fragment_start=0_end=88
LKNLK*KTLKQMTFCP*R*QKIDFLLCENIIIKTFYLRKLKFILKLKQK*NILKRDNWLKYSNF*LYNFKLSESSSRTRPSRKYATT*
RJVV01491319.1_start=0_end=30000_frame=- Fragment_start=0_end=267
AGCTGAAGAACTTGAAGTAGAAGACCTTGAAGCAGATGACGTTTTGCCCTTAGAGGTAACAAAAGATCGATTTTTTATTGtgtgaaaatataattataaaaacattttatttaagaaaattgaaatttattttaaaattaaaacaaaaataaaatattttaaaaagggaCAATTGGctaaaatattccaatttttaattatataattttaaactttcagagAGTTCGTCGAGAACCAGGCCGTCCAGGAAATATGCCACGACCTAAA
                                                                                                                        


# Proteins

In [14]:
records = get_augustus_proteins(annotation_file)
for idx, record in enumerate(records):
    print(idx, record.id, record.description)
    print(record.seq)
    print()

0 RJVV01031371.1_start=0_end=30000_frame=+ Gene_start=1393 Gene_end=3525 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLERLRREAEELEAEDLEADEVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSIIFIENE

1 RJVV01183868.1_start=0_end=30000_frame=- Gene_start=28 Gene_end=535 Auto
MPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSTIFIENE

2 RJVV01026129.1_start=0_end=30000_frame=+ Gene_start=1395 Gene_end=2603 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLESPSRTRPSRKYATT



# Commit proteins

In [21]:
def commit_proteins(proteins_file, new_records, force=False):
    all_proteins = {rec.id: rec for rec in SeqIO.parse(proteins_file, "fasta")}
    for record in new_records:
        if record.id in all_proteins and not force:
            warnings.warn(f"Protein {record.id} is already commited. Skipping...", category=UserWarning)
            continue
        all_proteins[record.id] = record
    return SeqIO.write(list(all_proteins.values()), proteins_file, "fasta")

## Commit auto

In [22]:
subset_to_commit = [0, 1, 2]
records_to_commit = [records[idx] for idx in subset_to_commit]
for idx, record in enumerate(records_to_commit):
    print(idx, record.id, record.description)
    print(record.seq)
    print()

0 RJVV01031371.1_start=0_end=30000_frame=+ Gene_start=1393 Gene_end=3525 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLERLRREAEELEAEDLEADEVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSIIFIENE

1 RJVV01183868.1_start=0_end=30000_frame=- Gene_start=28 Gene_end=535 Auto
MPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSTIFIENE

2 RJVV01026129.1_start=0_end=30000_frame=+ Gene_start=1395 Gene_end=2603 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLESPSRTRPSRKYATT



In [23]:
commit_proteins("hymenoptera_proteins.faa", records_to_commit)

3

## Commit custom

In [168]:
custom_record = SeqRecord(
    Seq(""),
    id="",
    description=f"Gene_start={} Gene_end={} Manual"
)
custom_record

SyntaxError: f-string: empty expression not allowed (1575694441.py, line 5)

In [ ]:
# commit_proteins("hymenoptera_proteins.faa", [custom_record])